### Forwards optimization notebook

By forward optimization here I mean optimization that only tunes existing parameters.
AKA finding the optimal value for any given parameter the strategy has so that it yields better results.

This notebook aims to create an interface json which can later be easily read for the optimal parameters for a given timeframe (start to date and period before to date).

for start_date in range(first_date, last_date):
    for end_date in range(start_date, last_date):
        generate_list of stuff
        
for thing in generated list not yet parsed:
    run the computation cronjob
    save the parsed config for later
    
def compute_optimal_stuff():
    filter the dataset & save it to disk maybe?
    groub by `each parameter` and apply function
    save the results for each parameter - function combination list
    
def save_otpimal_stuff()
    open the json
    save under date range key
    
results_json = {
    'date currently at': {
        '1-month': {
            'mean': {
                'sma': ..,
                'rsi close': ..
                ...
            },
            'order': ..
            'linear': ..
        }
        '3-month': ..
        '6-month': ..
        '12-month': ..
        'max': ..
    }
}

### Imports and variables

In [62]:
import datetime 
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd

# TIMEFRAME = 'daily'
TIMEFRAME = 'weekly'

FRAMEWORK = 'dask'
# FRAMEWORK = 'pandas'

# PERC_CHANGE_METHOD = 'mean'
PERC_CHANGE_METHOD = 'order aware'

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f'results/forward_opt_{TIMEFRAME}_results.json'

### Utils

In [63]:
def get_sample(the_df):
    print(f'There are {len(the_df)} rows in the dataframe/series')
    try:
        res = the_df.sample(frac=5)
    except ValueError:
        res = the_df.sample(3)
    
    return res

In [64]:
def save_results(key, results_for_key):
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        results_dict = json.loads(f.read())
        
    results_dict[key] = results_for_key
    
    with open(FORWARD_OPT_RESULTS_JSON, 'w') as f:
        f.write(json.dumps(results_dict))

# save_results('a', {1:2,3:4})

In [65]:
def get_pair_entry(start_date, end_date):
    return (
        f"{start_date.strftime('%y-%m')}-to-{end_date.strftime('%y-%m')}",
        start_date,
        end_date
    )

def generate_date_ranges():
    resulting_dict = {}
    for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE):
        resulting_dict[end_date.strftime('%y-%m')] = {
            '1-month': get_pair_entry(end_date - relativedelta(months=-1), end_date),
            '3-month': get_pair_entry(end_date - relativedelta(months=-3), end_date),
            '6-month': get_pair_entry(end_date - relativedelta(months=-6), end_date),
            '12-month': get_pair_entry(end_date - relativedelta(months=-12), end_date),
            'max': get_pair_entry(FIRST_DATE, end_date)
        }

    return resulting_dict


def get_parsed_date_ranges():
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        results_dict = json.loads(f.read())
    
    return list(results_dict.keys())

# for i in generate_date_ranges():
#     range_name, key, start_date, end_date = i
#     print(range_name, key, start_date, end_date)

In [ ]:
def mean_optimal_func(tmp_df_groupby):
    tmp_df = tmp_df_groupby["perc_change"].mean()
    max_value = tmp_df.idxmax()

    if FRAMEWORK == 'dask':
        max_value = max_value.compute()

    return max_value


def order_optimal_func(tmp_df_groupby):
    def outcome_computation(x):
        l = []
        for _, perc_change in x.sort_values(['date'])['perc_change'].iteritems():
            l.append(perc_change)

        results = []
        for i in range(len(l)):
            res = 1
            for j in range(i, len(l)):
                res += (res * l[j]) / 100
            results.append(res)

        return sum(results) / len(results)

    outcome_series = tmp_df_groupby.apply(outcome_computation, meta=('float'))    
    return outcome_series.idxmax().compute()


def linear_optimal_func(df):
    pass


def compute_optimal_params_using_func(df, func):
    """Common pattern where you group by a column and compute the mean percentage change."""
    tmp_df_groupby = df.groupby([column_name])       
    return 


def compute_optimal_results(df, start_date, end_date):
    filtered_df = df[(df['start_date'] >= start_date) & (df['end_date'] <= end_date)]
    return {
        'mean': compute_optimal_params_using_func(filtered_df, mean_optimal_func),
        'order': compute_optimal_params_using_func(filtered_df, order_optimal_func),
#         'linear': compute_optimal_params_using_func(filtered_df, linear_optimal_func),
        
    }


In [61]:
all_date_ranges = generate_date_ranges()
parsed_date_ranges = get_parsed_date_ranges()
unparsed_date_ranges = {k:v for k, v in all_date_ranges.items() if k not in parsed_date_ranges}

for date_value, date_ranges in unparsed_date_ranges.items():
    optimal_results = {}

    for period_range, values in date_ranges.items():
        _, start_date, end_date = values
        optimal_results[period_range] = compute_optimal_results(df, start_date, end_date)
        
    save_results(date_value, optimal_results)

1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1-month
3-month
6-month
12-month
max
1

### Load the dataframe

In [13]:
if TIMEFRAME == 'daily':
    path = 'dataset/daily/results'
elif TIMEFRAME == 'weekly':
    path = 'dataset/weekly/results'

if FRAMEWORK == 'pandas':
    d = pd
    f = f"{path}/A.csv"
elif FRAMEWORK == 'dask':
    d = dd
    f = f"{path}/A*.csv"

# Read CSV file and assign the right column names
# Comment the first two rows if you don't want to use the entire dataset, and instead just work on a sample
df = d.read_csv(
    f,
    names=[
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        #         'rsi_close_period',        # not for 'efficient_results'
        # 'days_ago_close_period'  # not for 'efficient_results'
        "adx8",
        "adx16",
        "adx32",
        "adx64",
        "adx128",
        "adx256",
        "adx512",
        "mdi8",
        "mdi16",
        "mdi32",
        "mdi64",
        "mdi128",
        "mdi256",
        "mdi512",
        "pdi8",
        "pdi16",
        "pdi32",
        "pdi64",
        "pdi128",
        "pdi256",
        "pdi512",
        "ppo8",
        "ppo16",
        "ppo32",
        "ppo64",
        "ppo128",
        "ppo256",
        "ppo512",
        "stochastic8",
        "stochastic16",
        "stochastic32",
        "stochastic64",
        "stochastic128",
        "stochastic256",
        "stochastic512",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ],
)

In [14]:
get_sample(df)

There are 77175 rows in the dataframe/series


,uid,ticker,date,price_open,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,adx128,adx256,adx512,mdi8,mdi16,mdi32,mdi64,mdi128,mdi256,mdi512,pdi8,pdi16,pdi32,pdi64,pdi128,pdi256,pdi512,ppo8,ppo16,ppo32,ppo64,ppo128,ppo256,ppo512,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,price_sell,days_ago_close_period,rsi_close_period
npartitions=14,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Compute the change percentage
And drop the price_sell and price_open columns because they are no longer used

In [15]:
# Add the percentage change for the price
df["perc_change"] = (df["price_sell"] - df["price_open"]) / df["price_open"] * 100

In [16]:
# The prices can now be dropped since I won't need them anymore
df = df.drop(["price_sell", "price_open"], axis=1)

In [17]:
get_sample(df)

There are 77175 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,adx128,adx256,adx512,mdi8,mdi16,mdi32,mdi64,mdi128,mdi256,mdi512,pdi8,pdi16,pdi32,pdi64,pdi128,pdi256,pdi512,ppo8,ppo16,ppo32,ppo64,ppo128,ppo256,ppo512,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,days_ago_close_period,rsi_close_period,perc_change
npartitions=14,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Strategy explanation
## Initial strategy parameters
### SMA
The simple moving average of close prices for the close price of the previous day to be above so that we have the first buy signal
### RSI open
The relative strength index lower barrier to break for a buy signal
### RSI close
The relative strength index barrier that, if it breaks above, indicates a sell signal
### Days to wait before closing a trade
If the RSI close barrier is not reached, how long to hold the trade before selling.

## Forward optimizations goals 
Find the best parameter combination for this strategy by exploring relatively close range of values for each of the previously mentioned parameters.
It's called 'forward' because we already have a strategy defined but we just tweak it's parameters in the hopes of improving results.
This will be explored next, and the focus on 'backward' optimization will fall into a later part of the notebook.


##### It is important to note how the dataset was generated, only the RSI open and SMA indicators were considered and 20 days were recorded.

### Optimization utilities
The idea is simple:

    1. Groub py a field
    2. Find the best value for 'perc_change'
    3. Return the best value for the field that was grouped by
    
The problem is step 2. How do you compute the best perc_change value?
Ideas:

    * The simplest approach is to use the mean. It's easy to compute but doesn't show us the real picture because it doesn't account for the order of trades                     

    * Another option may be to apply these trades in order to one unit. This doesn't account for random starting times.                                  
    
    * Another option may be to apply these trader in order to one unit, multiple times, each time dropping one entry from beggining to end. Finally, find the mean of these resulting values. The advantage is that it accounts for order but also for random starting points.                       

In [18]:
def outcome_computation(x):
    l = []
    for _, perc_change in x.sort_values(['date'])['perc_change'].iteritems():
        l.append(perc_change)

    results = []
    for i in range(len(l)):
        res = 1
        for j in range(i, len(l)):
            res += (res * l[j]) / 100
        results.append(res)

    return sum(results) / len(results)

In [19]:
def grouby_column_get_best_perc_change(the_df, column_name):
    """Common pattern where you group by a column and compute the mean percentage change."""
    tmp_df_groupby = df.groupby([column_name])   

    if PERC_CHANGE_METHOD == 'mean':
        tmp_df = tmp_df_groupby["perc_change"].mean()
        max_value = tmp_df.idxmax()

        if FRAMEWORK == 'dask':
            max_value = max_value.compute()
        
        print(tmp_df)

    elif PERC_CHANGE_METHOD == 'order aware':
        outcome_series = tmp_df_groupby.apply(outcome_computation, meta=('float'))    
        max_value = outcome_series.idxmax().compute()
        
        print(outcome_series)
        
    print(f'Seems like the best value for `{column_name}` is {max_value}')
    
    return max_value

### 1. Find the optimal number of days to wait before closing a trade

In [9]:
# Group by it and find the mean percentage change
max_dacp_value = grouby_column_get_best_perc_change(df, 'days_ago_close_period')

Dask Series Structure:
npartitions=52
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 782 tasks
Seems like the best value for `days_ago_close_period` is 20


In [10]:
# Keep only rows where it is optimal
df = df[df['days_ago_close_period'] == max_dacp_value]

In [11]:
get_sample(df)

There are 31589 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,adx128,adx256,adx512,mdi8,mdi16,mdi32,mdi64,mdi128,mdi256,mdi512,pdi8,pdi16,pdi32,pdi64,pdi128,pdi256,pdi512,ppo8,ppo16,ppo32,ppo64,ppo128,ppo256,ppo512,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,days_ago_close_period,rsi_close_period,perc_change
npartitions=52,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### 2. Find the optimal RSI open barrier to open a trade

In [12]:
# Group by it and find the mean percentage change
max_rsicp_value = grouby_column_get_best_perc_change(df, 'rsi_open_period')

Dask Series Structure:
npartitions=52
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 938 tasks
Seems like the best value for `rsi_open_period` is 6


In [13]:
# Keep only rows where it is optimal
df = df[df['rsi_open_period'] == max_rsicp_value]

In [14]:
get_sample(df)

There are 13597 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,adx128,adx256,adx512,mdi8,mdi16,mdi32,mdi64,mdi128,mdi256,mdi512,pdi8,pdi16,pdi32,pdi64,pdi128,pdi256,pdi512,ppo8,ppo16,ppo32,ppo64,ppo128,ppo256,ppo512,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,days_ago_close_period,rsi_close_period,perc_change
npartitions=52,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### 3. Find the optimal SMA to open a trade

In [15]:
# Group by it and find the mean percentage change
max_sma_value = grouby_column_get_best_perc_change(df, 'maperiod')

Dask Series Structure:
npartitions=52
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 1094 tasks
Seems like the best value for `maperiod` is 220


In [16]:
# Keep only rows where it is optimal
df = df[df['maperiod'] == max_sma_value]

In [225]:
get_sample(df)

There are 3 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,adx128,adx256,adx512,mdi8,mdi16,mdi32,...,ppo32,ppo64,ppo128,ppo256,ppo512,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,days_ago_close_period,rsi_close_period,perc_change
2445,714111d6-ad7a-11eb-bbf8-00155de4f607,A,2017-11-10,180,6,39.677175,32.894764,27.090050,24.556437,16.307752,7.818978,3.629262,24.409508,18.918690,17.204782,...,11.093096,8.534682,3.570242,-4.390091,-14.233002,34.132196,46.188495,76.335362,87.921079,90.970072,95.406824,96.577687,6,80.355072,4.640049
1208,6f76e7ae-ad7a-11eb-bbf8-00155de4f607,A,2007-04-30,180,6,36.063133,30.591766,17.519740,9.469102,6.896313,5.486014,2.122430,28.773607,20.718881,18.726114,...,4.756449,2.885483,1.043431,-1.469326,-6.338171,21.865013,31.045752,75.321637,80.624426,80.624426,78.592879,86.067943,6,83.769682,8.921438
688,6f024f70-ad7a-11eb-bbf8-00155de4f607,A,2005-08-08,180,6,31.674421,29.963212,23.194669,11.943594,5.609116,2.715142,1.201641,32.061420,23.869667,20.222260,...,6.372101,3.119495,0.398767,0.200676,3.539854,19.385216,34.408602,74.261603,82.485389,85.194175,86.414254,32.577666,6,89.648795,22.826087


### 4. Find the optimal RSI close barrier to close a trade
This is a bit different because RSI close is logged as a float, so we have to check for a bin instead.
This is also last because it takes a LOT of time, even for small dataframes?

In [23]:
outcome_computation(df[df['rsi_close_period'] >= threshold].compute())

2498154362847.0747

In [26]:
max_value = 0
optimal_threshold = 10
for threshold in range(10, 101, 10):
    if PERC_CHANGE_METHOD == 'mean':
        result = df[df['rsi_close_period'] >= threshold]['perc_change'].mean()
        result = result.compute() if FRAMEWORK == 'dask' else result 
        print(threshold, result)
    elif PERC_CHANGE_METHOD == 'order aware':
        result = outcome_computation(df[df['rsi_close_period'] >= threshold])
        print(threshold, result)
    
    
    if result > max_value:
        optimal_threshold, max_value = threshold, result

print(f'Seems like the best threshold for rsi_close_period is {optimal_threshold}')

10 2568688634596.0522
20 1689147933515865.0
30 3.4255230492819714e+19
40 3.0710504335771495e+23
50 2.3457585259018034e+25
60 5.707342182472024e+22
70 4457824206702118.5
80 3641943.836032747
90 6.739141771599958


ZeroDivisionError: division by zero

In [27]:
# Keep only rows where it is optimal
df = df[df['rsi_close_period'] >= 50]#optimal_threshold]

In [222]:
get_sample(df)

There are 18 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,adx128,adx256,adx512,mdi8,mdi16,mdi32,...,ppo32,ppo64,ppo128,ppo256,ppo512,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,days_ago_close_period,rsi_close_period,perc_change
1208,6f76e7ae-ad7a-11eb-bbf8-00155de4f607,A,2007-04-30,180,6,36.063133,30.591766,17.519740,9.469102,6.896313,5.486014,2.122430,28.773607,20.718881,18.726114,...,4.756449,2.885483,1.043431,-1.469326,-6.338171,21.865013,31.045752,75.321637,80.624426,80.624426,78.592879,86.067943,6,83.769682,8.921438
2337,71238a12-ad7a-11eb-b0f6-00155de4f607,A,2005-08-08,200,6,31.674421,29.963212,23.194669,11.943594,5.609116,2.715142,1.201641,32.061420,23.869667,20.222260,...,6.358619,3.106425,0.386042,0.187976,3.526731,19.385216,34.408602,74.261603,82.485389,85.194175,86.414254,32.577666,6,89.648795,22.826087
9883,7a72b138-ad7a-11eb-973e-00155de4f607,A,2017-11-10,240,6,39.677175,32.894764,27.090050,24.556437,16.307752,7.818978,3.629262,24.409508,18.918690,17.204782,...,15.171905,12.519558,7.372848,-0.879751,-11.084046,34.132196,46.188495,76.335362,87.921079,90.970072,95.406824,96.577687,6,80.355072,4.640049


## Save the results for later use
These optimizations shouldn't be run every time, we should be able to start only from the next section by reading ths df csv

In [28]:
# We won't need the fixed parameters from now on so I drop them here
df = df.drop(['maperiod', 'rsi_open_period', 'rsi_close_period', 'days_ago_close_period'], axis=1)

In [29]:
if FRAMEWORK == 'dask':
    path = 'forward_opt_results'
else:
    path = 'forward_opt_results.csv'

df.to_csv(path)

['/home/narboom23/Projects/licenta/forward_opt_results/00.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/01.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/02.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/03.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/04.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/05.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/06.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/07.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/08.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/09.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/10.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/11.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/12.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/13.part',
 '/home/narboom23/Projects/licenta/forward_opt_results/14.part',
 '/home/narboom23/Project

### Year by year optimizations
How do these optimal parameters change on a yearly basis?

In [11]:
years = ['2000', 
'2001', 
'2002', 
'2003', 
'2004', 
'2005', 
'2006', 
'2007', 
'2008', 
'2009', 
'2010', 
'2011', 
'2012', 
'2013', 
'2014', 
'2015', 
'2016', 
'2017', 
'2018', 
'2019',
'2020'] 

In [11]:
for year in years:
    # Restrict the dataframe
    tmp_df = df[df['date'].str.contains(year)]
    
    # Group by it and find the mean percentage change
    max_dacp_value = grouby_column_get_best_perc_change(tmp_df, 'days_ago_close_period')
    
    print(max_dacp_value)

Dask Series Structure:
npartitions=52
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 782 tasks
Seems like the best value for `days_ago_close_period` is 20
20
Dask Series Structure:
npartitions=52
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 782 tasks
Seems like the best value for `days_ago_close_period` is 20
20
Dask Series Structure:
npartitions=52
    float64
        ...
     ...   
        ...
        ...
dtype: float64
Dask Name: outcome_computation, 782 tasks
Seems like the best value for `days_ago_close_period` is 20
20


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/narboom23/Projects/licenta/env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-c3d052ff18ec>", line 6, in <module>
    max_dacp_value = grouby_column_get_best_perc_change(tmp_df, 'days_ago_close_period')
  File "<ipython-input-8-7ac28b470034>", line 16, in grouby_column_get_best_perc_change
    max_value = outcome_series.idxmax().compute()
  File "/home/narboom23/Projects/licenta/env/lib/python3.9/site-packages/dask/base.py", line 285, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/home/narboom23/Projects/licenta/env/lib/python3.9/site-packages/dask/base.py", line 567, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/home/narboom23/Projects/licenta/env/lib/python3.9/site-packages/dask/threaded.py", line 79, in get
    results = get_async(
  File "/home/narboom23/Projects/licenta


KeyboardInterrupt



In [12]:
max_dacp_value

ERROR! Session/line number was not unique in database. History logging moved to new session 30


20

'2000', 
'2001', 
'2002', 
'2003', 
'2004', 
'2005', 
'2006', 
'2007', 
'2008', 
'2009', 
'2010', 
'2011', 
'2012', 
'2013', 
'2014', 
'2015', 
'2016', 
'2017', 
'2018', 
'2019', 
